In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from custom_trainer import CustomTrainer
from custom_dataset import build_dataloader
from custom_model import ConvTransformerAE
import torch
import random
import numpy as np
torch.manual_seed(0)
random.seed(0)

In [ ]:
data_path = './resources/features_norm.npy'
label_path = './resources/labels.npy'
train_loader = build_dataloader(
    data_path=data_path,
    label_path=label_path,
    batch_size=4096,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    drop_last=True
)

model = ConvTransformerAE(
    input_dim=32,
    latent_dim=16,
    conv_channels=4,
    d_model=32,
    n_head=4,
    num_layers=3
)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=1000,   # 每 10 个 epoch 衰减一次
    gamma=0.95       # lr *= 0.5
)
trainer = CustomTrainer(
    model=model,
    optimizer=optimizer,
    device='cuda',
    scheduler=scheduler,
)

In [ ]:
trainer.train(train_loader, epochs=5)